In [0]:
#This code takes two GAP groups and determines whether they form a Brauer pair or have same outer char table.

from collections import defaultdict

def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return set(factors)

def FirstProperties(G):
    """input a GAP group and return list of properties"""

    #conj_class_ele_order = [gap.Order(gap.Representative(c)) for c in gap.ConjugacyClasses(G)]
    #conj_class_ele_order.sort()

    #size_of_conj_classes = [gap.Size(c) for c in gap.ConjugacyClasses(G)]
    #size_of_conj_classes.sort()

    size_and_order_of_conj_classes = [(gap.Size(c),gap.Order(gap.Representative(c))) for c in gap.ConjugacyClasses(G)]
    size_and_order_of_conj_classes.sort()

    C = gap.Centre(G)
    center = gap.IdGroup(C) if gap.Size(C) < 500 else gap.Size(C)
    derived = gap.DerivedSubgroup(G)
    F = gap.FactorGroup(G, derived)
    abelianisation = gap.IdGroup(F) if gap.Size(F) < 500 else gap.Size(F)

    return (tuple(size_and_order_of_conj_classes), center, abelianisation)

def BrauerPairs(n=1):
    """Given an order n, determines if any two or more groups form a Brauer Pair (or tuple)"""

    fail = gap.TransformingPermutationsCharacterTables(gap.CharacterTable(gap.SmallGroup(2,1)),gap.CharacterTable(gap.SmallGroup(3,1)))

    GroupsSortedByFirstProperties = defaultdict(list)

    k = 1
    while k >0:
        try:
            G = gap.SmallGroup(n,k)
        except RuntimeError:
            break
        if gap.Size(gap.Centre(G)) == n:
            pass
        else:
            GroupsSortedByFirstProperties[FirstProperties(G)].append(k)
        k +=1

    for key,value in GroupsSortedByFirstProperties.items():
        if len(value) > 1: #i.e. there are two or more k values with same "First Properties".
            BrauerPair = defaultdict(list)

            index_already_appended = []
            for i in range(len(value)):
                if i in index_already_appended:
                    pass
                elif i == len(value) -1:
                    #If last element of value list is not already in the dict then it must go under its own key in the dict
                    BrauerPair[value[-1]] = [value[-1]]
                else:
                    BrauerPair[value[i]] = [value[i]] #element with index i obviously has same value under Property_func as itself.
                    for j in range(i+1,len(value)):
                        G = gap.SmallGroup(n,value[i])
                        H = gap.SmallGroup(n,value[j])
                        if gap.TransformingPermutationsCharacterTables(gap.CharacterTable(G),gap.CharacterTable(H)) != fail:
                            BrauerPair[value[i]].append(value[j])
                            index_already_appended.append(j)
            for b_value in BrauerPair.values():
                if len(b_value) > 1:
                    print("A Brauer Pair has been found!")
                    print(f"n is {n}, list of k values which form Brauer pair is {b_value}")
                    print("")

def BrauerPairsSmallPrimeExponent(start = 1):
    """Code to attempt to find Brauer Pairs of order n where 2^8 !| n, 3^6 !| n, p^5 !| n for primes p > 3.
    Vaguely, n with small prime exponents"""
    n = start
    while n >0:
        if n % 5 ==0:
            print(f"groups of order < {n} have been checked.")
            print("")
        bigexpo = False
        if n % 2**8 == 0 or n % 3**6 == 0:
            bigexpo = True
        else:
            for prime in prime_factors(n):
                if prime != 2 and prime != 3:
                    if n % prime**5 == 0:
                        bigexpo = True
        if bigexpo == True:
            print(f"{n} has large prime exponent and will be skipped")
            print("")
        else:
            BrauerPairs(n)
        n +=1

#BrauerPairs()
#BrauerPairsSmallPrimeExponent()